# Transformed Hourly Weather Data
Author: Colin Pannikkat

This notebook transforms the Posch et. al hourly weather data into a usable input file for the GARISOM model. Soil temperature was not provided, and is instead retrieved from NLDAS in file_builder.py when building the simulation files.

In [115]:
import pandas as pd
from datetime import datetime
import numpy as np

In [116]:
hourly_weather = pd.read_csv("230926_DBG_cottonwood_weather_clean.csv")
hourly_weather.head()

,TIMESTAMP,RECORD,DateTime(1),DateTime(9),DateTime(4),DateTime(5),Target,BattVolt_Avg,Pnl_Tmp_Avg,WindSpeed_Avg,...,TbodyC_Avg(1),TbodyC_Avg(2),TtarRC_Avg,TtarHC_Avg,TadjH_Avg,T_Diffrns_Avg,PID_out_Avg,PID_lmt_Avg,ScldOut_Avg,TrelayBox_Avg
0,2023-07-20 09:10:00,0,2023,201,9,10,3,13.58706,40.57328,0.591333,...,NAN,-273.15,NAN,NAN,NAN,NAN,NAN,NAN,NAN,39.85889
1,2023-07-20 09:15:00,1,2023,201,9,15,3,13.58919,40.83521,1.151000,...,NAN,-273.15,NAN,NAN,NAN,NAN,NAN,NAN,NAN,40.31171
2,2023-07-20 09:20:00,2,2023,201,9,20,3,13.56165,41.22749,0.845133,...,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,40.81530
3,2023-07-20 09:25:00,3,2023,201,9,25,3,13.57002,41.45853,1.050300,...,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,40.10521
4,2023-07-20 09:30:00,4,2023,201,9,30,3,13.55727,41.29865,0.952267,...,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,40.91309


In [117]:
hourly_weather = hourly_weather[['TIMESTAMP', 'WindSpeed_Avg', 'AirT_Avg', 'AirRH_Avg']]

In [118]:
hourly_weather['TIMESTAMP'] = hourly_weather['TIMESTAMP'].apply(
    lambda x: (
        pd.to_datetime(x[:10]) + pd.Timedelta(days=1)
    ).strftime("%Y-%m-%d 00:00:00") if x[11:19] == "24:00:00" else x
)
hourly_weather['TIMESTAMP'] = pd.to_datetime(hourly_weather['TIMESTAMP'], format="%Y-%m-%d %H:%M:%S")

In [119]:
hourly_weather = hourly_weather[hourly_weather['TIMESTAMP'].dt.minute == 0]

In [120]:
hourly_weather.head()

,TIMESTAMP,WindSpeed_Avg,AirT_Avg,AirRH_Avg
10,2023-07-20 10:00:00,0.231200,39.03725,18.27692
22,2023-07-20 11:00:00,1.828333,40.40611,15.85820
34,2023-07-20 12:00:00,2.153666,41.97869,14.09526
46,2023-07-20 13:00:00,2.295000,43.51690,11.34986
58,2023-07-20 14:00:00,2.767000,44.13353,10.22450


In [121]:
par_and_rain_data = pd.read_csv("weather_community_garden_211029-240630.csv")
par_and_rain_data.head()

,No.,Date.Time,Rain.mm,Wind.Direction,Temp.C,RH.percent,Wind.Speed.m.s,Gust.Speed.m.s,PAR.mumol.m2.s,Notes
0,1,10/29/2021 10:28,0.0,70.2,28.221,19.2,2.52,6.04,1973.7,Plot Title: Community garden
1,2,10/29/2021 10:58,0.0,71.6,29.090,18.6,2.52,5.03,1406.2,"Date Time, GMT-07:00"
2,3,10/29/2021 11:28,0.0,68.8,30.318,17.3,2.01,4.53,1466.2,"Rain, mm (LGR S/N: 10844781, SEN S/N: 10817995)"
3,4,10/29/2021 11:58,0.0,71.6,31.001,16.2,2.01,5.03,1496.2,"Wind Direction, ø (LGR S/N: 10844781, SEN S/N:..."
4,5,10/29/2021 12:28,0.0,77.2,31.459,15.7,1.51,4.03,1491.2,"Temp, °C (LGR S/N: 10844781, SEN S/N: 10830566..."


In [122]:
par_and_rain_data['Date.Time'] = pd.to_datetime(par_and_rain_data['Date.Time'], format="%m/%d/%Y %H:%M")

In [123]:
def round_up_to_half_hour(dt):
    minute = dt.minute
    if minute == 0:
        return dt
    elif minute <= 30:
        return dt.replace(minute=30, second=0, microsecond=0)
    else:
        return (dt + pd.Timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

In [124]:
# Round TIMESTAMP in hourly_weather up to the nearest half hour
par_and_rain_data['Date.Time'] = par_and_rain_data['Date.Time'].apply(round_up_to_half_hour)
par_and_rain_data = par_and_rain_data.rename(columns={'Date.Time' : 'TIMESTAMP'})

In [125]:
par_and_rain_data = par_and_rain_data[par_and_rain_data['TIMESTAMP'].dt.minute == 0]

In [126]:
# Filter for 07/20/2023 and hours 00:00 to 09:00
par_and_rain_data_morning = par_and_rain_data[
    (par_and_rain_data['TIMESTAMP'].dt.date == pd.to_datetime('2023-07-20').date()) &
    (par_and_rain_data['TIMESTAMP'].dt.hour.between(0, 9))
]
par_and_rain_data_morning = par_and_rain_data_morning.rename(
    columns={
        'Wind.Speed.m.s': 'WindSpeed_Avg',
        'Temp.C': 'AirT_Avg',
        'RH.percent': 'AirRH_Avg'
    }
)
par_and_rain_data_morning.head()

,No.,TIMESTAMP,Rain.mm,Wind.Direction,AirT_Avg,AirRH_Avg,WindSpeed_Avg,Gust.Speed.m.s,PAR.mumol.m2.s,Notes
30171,30172,2023-07-20 00:00:00,0.25,105.3,36.498,24.7,1.01,3.02,1.2,NaN
30173,30174,2023-07-20 01:00:00,0.25,94.1,34.624,26.1,0.50,3.52,1.2,NaN
30175,30176,2023-07-20 02:00:00,0.25,103.9,30.874,29.9,0.00,0.50,1.2,NaN
30177,30178,2023-07-20 03:00:00,0.25,103.9,30.925,31.1,0.00,1.01,1.2,NaN
30179,30180,2023-07-20 04:00:00,0.25,103.9,30.293,33.9,0.00,0.00,1.2,NaN


In [127]:
hourly_weather = pd.concat([
    par_and_rain_data_morning[['TIMESTAMP', 'WindSpeed_Avg', 'AirT_Avg', 'AirRH_Avg']],
    hourly_weather
], ignore_index=True, sort=False)

In [128]:
par_and_rain_data = par_and_rain_data[["TIMESTAMP", "Rain.mm", "PAR.mumol.m2.s"]]

In [129]:
hourly_weather = hourly_weather.merge(par_and_rain_data, how='left', on='TIMESTAMP')

In [130]:
water_amount = pd.read_csv("./water amounts_pots_2023.csv")
water_amount.head()

,Date.yymmdd,DOY,Time.Start,Time.Stop,irrigation.mm3,irrigation.mm,precipitation.mm
0,230527,147,0:00:00,0:01:00,0.0,0.0,0.0
1,230527,147,0:01:00,0:02:00,0.0,0.0,0.0
2,230527,147,0:02:00,0:03:00,0.0,0.0,0.0
3,230527,147,0:03:00,0:04:00,0.0,0.0,0.0
4,230527,147,0:04:00,0:05:00,0.0,0.0,0.0


In [131]:
water_amount['Year'] = water_amount['Date.yymmdd'].map(lambda x: datetime.strptime(str(x), "%y%m%d").strftime("%Y"))

In [132]:
water_amount['Day'] = water_amount['DOY'].map(lambda x: datetime.strptime(str(x), "%j").strftime("%-j"))

In [133]:
water_amount['Hour'] = water_amount['Time.Start'].map(lambda x: datetime.strptime(x, "0:%H:%M").strftime("%H"))

In [134]:
new_hourly_weather = pd.DataFrame(columns=['Year', 'Day', 'Hour', 'Solar_Wm2', 'Rain_mm', 'Wind_ms.1', 'Tair_C', 'D_kPa'])

In [135]:
new_hourly_weather['Rain_mm'] = hourly_weather['Rain.mm'].astype(float)

In [136]:
new_hourly_weather['Wind_ms.1'] = hourly_weather['WindSpeed_Avg']

In [137]:
new_hourly_weather['Tair_C'] = hourly_weather['AirT_Avg'].astype(float)

In [138]:
new_hourly_weather['Year'] = hourly_weather['TIMESTAMP'].map(lambda x: x.strftime("%Y"))

In [139]:
new_hourly_weather['Day'] = hourly_weather['TIMESTAMP'].map(lambda x: x.strftime("%-j"))

In [140]:
new_hourly_weather['Hour'] = hourly_weather['TIMESTAMP'].map(lambda x: x.strftime("%H"))

In [141]:
new_hourly_weather.set_index(['Year','Day','Hour'])['Rain_mm']

Year  Day  Hour
2023  201  00      0.25
           01      0.25
           02      0.25
           03      0.25
           04      0.25
                   ... 
      269  05      0.00
           06      0.00
           07      0.00
           08      0.00
           09      0.00
Name: Rain_mm, Length: 1642, dtype: float64

In [142]:
water_amount.set_index(['Year', 'Day', 'Hour'])['irrigation.mm']

Year  Day  Hour
2023  147  00      0.0
           01      0.0
           02      0.0
           03      0.0
           04      0.0
                  ... 
      301  19      0.0
           20      0.0
           21      0.0
           22      0.0
           23      0.0
Name: irrigation.mm, Length: 3696, dtype: float64

In [143]:
# Align indices before addition
water_amount_indexed = water_amount.set_index(['Year', 'Day', 'Hour'])['irrigation.mm'] #/ 10
new_hourly_weather_indexed = new_hourly_weather.set_index(['Year', 'Day', 'Hour'])['Rain_mm']

# Reindex water_amount_indexed to match new_hourly_weather_indexed
water_amount_indexed = water_amount_indexed.reindex(new_hourly_weather_indexed.index, fill_value=0)

# Perform addition with aligned indices
new_hourly_weather['Rain_mm'] = water_amount_indexed.add(new_hourly_weather_indexed, fill_value=0).reset_index(drop=True)

In [144]:
new_hourly_weather.head()

,Year,Day,Hour,Solar_Wm2,Rain_mm,Wind_ms.1,Tair_C,D_kPa
0,2023,201,00,NaN,63.45,1.01,36.498,NaN
1,2023,201,01,NaN,0.25,0.50,34.624,NaN
2,2023,201,02,NaN,0.25,0.00,30.874,NaN
3,2023,201,03,NaN,0.25,0.00,30.925,NaN
4,2023,201,04,NaN,0.25,0.00,30.293,NaN


In [145]:
# Add irrigation values for pre May 27th twice a day, 6.32mm
new_hourly_weather.loc[
    (new_hourly_weather['Day'].astype(int) < 147) & 
    ((new_hourly_weather['Hour'].astype(int) == 6) | (new_hourly_weather['Hour'].astype(int) == 18)),
    'Rain_mm'
] += 63.2 # / 10

In [146]:
def calc_e_water(T):
    '''
    Calculate saturation vapor pressure for water based on Buck (1996).
    '''
    return 6.1121 * np.exp((18.678 - (T / 234.5)) * (T / (257.14 + T))) * 0.1  # 1 hPa to 0.1 kPa
def calc_e_ice(T):
    '''
    Calculate saturation vapor pressure for ice based on Buck (1996).
    '''
    return 6.1115 * np.exp((23.036 - (T / 333.7)) * (T / (279.824 + T))) * 0.1  # 1 hPa to 0.1 kPa

In [147]:
def calc_vpd(air_temp, rh, saturation_vapor_pressure):
    '''
    Calculates VPD according to saturation vapor pressure calculations of Buck 
    (1996), these are modifications of Buck (1981) that does not require an 
    enhancement factor specification.

    VPD = e_s * (1 - RH/100)
    e_s is dependent on whether T > 0 or < 0

    air_temp must be in C, rh in percent, saturation_vapor_pressure uses Buck
    calculations which returns kPa after adjustment.
    '''
    return (saturation_vapor_pressure(air_temp) * (1 - (rh / 100)))

In [148]:
# Calculate VPD in kPa
new_hourly_weather['D_kPa'] = hourly_weather.apply(
    lambda row: calc_vpd(row['AirT_Avg'], row['AirRH_Avg'], calc_e_water) if row['AirT_Avg'] > 0 else calc_vpd(row['AirT_Avg'], row['AirRH_Avg'], calc_e_ice),
    axis=1
)

In [149]:
def convert_par_to_solar_radiation(par):
    '''
    Conversion done per:

    Reis, Mariana & Ribeiro, Aristides. (2020). Conversion factors and general 
    quations applied in agricultural and forest meteorology. 27. 227-258. 
    10.31062/agrom.v27i2.26527. 

    "The approximation 1 W m-2 ≈ 4.57 μmol m-2 s-1 (Thimijan & Heins, 1983) is 
    assuming that the W m-2 is for photosynthetically active radiation (PAR) 
    from 4.0 to 7.0 µm."

    Sensor used for cottonwood data was HOBO S-LIA-M003, which measures
    between 400 to 700 nm, so this is fine to use, but for other sensors that do
    not measure in that range, PAR is ~2.02 instead.
    '''
    return par / 4.57

In [150]:
# Subtract weird baseline (1.2) and calculate solar radiation in Wm^-2 from micromoles/m2/s
new_hourly_weather['Solar_Wm2'] = hourly_weather['PAR.mumol.m2.s'].apply(lambda x: x - 1.2).apply(convert_par_to_solar_radiation)

In [151]:
# Limit data to only within the experimental predrought and post drought time periods
new_hourly_weather = new_hourly_weather.loc[(new_hourly_weather['Day'].astype(int)>= 201) & (new_hourly_weather['Day'].astype(int) <= 265)]

In [152]:
new_hourly_weather.to_csv("../dataset.csv", index=False)